In [1]:
!pip install prophet


In [4]:
import pandas as pd
from prophet import Prophet

# Load dataset
df = pd.read_csv('HadCRUT4.csv')

# Combine 'Year' and 'Month' into a single datetime column
df['Day'] = 1  # Add a dummy day
df['ds'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df = df.rename(columns={'Anomaly': 'y'})

# Filter historical data until Dec 2020
historical_df = df[df['ds'] <= '2020-12-31'][['ds', 'y']]

# Initialize and train Prophet model
model = Prophet(yearly_seasonality=True)
model.fit(historical_df)

# Forecast from Jan 2021 to Dec 2100
future = model.make_future_dataframe(periods=80*12, freq='MS')  # 80 years * 12 months
forecast = model.predict(future)

# Filter predictions from 2021 onwards
future_forecast = forecast[['ds', 'yhat']].copy()
future_forecast = future_forecast[future_forecast['ds'] >= '2021-01-01']

# Extract Year and Month, round to 3 decimal places
future_forecast['Year'] = future_forecast['ds'].dt.year
future_forecast['Month'] = future_forecast['ds'].dt.month
future_forecast['Predicted_Anomaly'] = future_forecast['yhat'].round(3)

# Select and reorder columns
output_df = future_forecast[['Year', 'Month', 'Predicted_Anomaly']]

# Save to CSV
output_df.to_csv('predicted_anomalies_monthly_2021_2100.csv', index=False)

print("Prediction complete. File saved as 'predicted_anomalies_monthly_2021_2100.csv'.")


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpq8p032gs/uojmq73z.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpq8p032gs/_o_k6s53.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=46794', 'data', 'file=/tmp/tmpq8p032gs/uojmq73z.json', 'init=/tmp/tmpq8p032gs/_o_k6s53.json', 'output', 'file=/tmp/tmpq8p032gs/prophet_model6lsedtbg/prophet_model-20250323073956.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:39:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:39:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prediction complete. File saved as 'predicted_anomalies_monthly_2021_2100.csv'.


In [9]:
import pandas as pd
from prophet import Prophet

# Load dataset
df = pd.read_csv("temp-1901-2020-all.csv")
print("🔍 Columns in CSV:", df.columns.tolist())  # Debug check

# Extract month abbreviation from 'Statistics' (e.g., "Jan Average")
df['Month'] = df['Statistics'].str.slice(0, 3)

# Map month abbreviations to numeric values
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}
df['Month_num'] = df['Month'].map(month_map)

# Add 'Day' column and datetime for Prophet
df['Day'] = 1
df['ds'] = pd.to_datetime(dict(year=df['Year'], month=df['Month_num'], day=df['Day']))

# Rename Temperature to y for Prophet
df = df.rename(columns={'Temperature': 'y'})

# Prophet model
model = Prophet()
model.fit(df[['ds', 'y']])

# Predict from 2021 to 2100 (monthly)
future = model.make_future_dataframe(periods=(2100 - 2020) * 12, freq='MS')
forecast = model.predict(future)

# Filter only future predictions
forecast_filtered = forecast[forecast['ds'].dt.year > 2020].copy()

# Round temperature to 1 decimal
forecast_filtered['Temperature'] = forecast_filtered['yhat'].round(1)

# Extract Year and Month abbreviation
forecast_filtered['Year'] = forecast_filtered['ds'].dt.year
forecast_filtered['Month'] = forecast_filtered['ds'].dt.strftime('%b')

# Create 'Statistics' as "Jan Average", etc.
forecast_filtered['Statistics'] = forecast_filtered['Month'] + ' Average'

# Add constant Country and ISO3 values
forecast_filtered['Country'] = 'Australia'
forecast_filtered['ISO3'] = 'AUS'

# Final output format
forecast_final = forecast_filtered[['Temperature', 'Year', 'Statistics', 'Country', 'ISO3']]

# Save to CSV
forecast_final.to_csv('temperature_predictions_2021_2100.csv', index=False)

print("✅ Saved: 'temperature_predictions_2021_2100.csv'")


🔍 Columns in CSV: ['Temperature', 'Year', 'Statistics', 'Country', 'ISO3']


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpq8p032gs/senqnmdz.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpq8p032gs/ate4r30t.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=52745', 'data', 'file=/tmp/tmpq8p032gs/senqnmdz.json', 'init=/tmp/tmpq8p032gs/ate4r30t.json', 'output', 'file=/tmp/tmpq8p032gs/prophet_model5lfpgouw/prophet_model-20250323075614.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:56:14 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:56:37 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


✅ Saved: 'temperature_predictions_2021_2100.csv'
